In [2]:
# https://github.com/arditobryan/datasets/tree/main/220226_steam 참고
import random
import requests
import time
from datetime import datetime

import pandas as pd
from tqdm import tqdm

In [ ]:
# 0. Functions
def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})

        # Check if response is JSON (Error Handling)
        try:
            return response.json()
        except requests.exceptions.JSONDecodeError:
            print("Warning: Non-JSON response received, possibly due to rate limiting or server issue.")
            return None  # Return None if the response isn't JSON


def get_n_reviews(appid, language="english", num_try=None, start_date=None, end_date=None, sleep=1):
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language': language,
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }

    cnt = 0
    empty_response_count = 0
    previous_reviews_num = 0
    max_empty_responses = 50


    if start_date != None:
        start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
    else:
        start_timestamp = None
    if end_date != None:
        end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())
    else:
        end_timestamp = None


    while True:
        cnt += 1

        # If you want to limit the number of tries
        if num_try != None:
            if cnt == num_try:
                print(f"Reached {num_try}th try.\nNum. of reviews: {len(reviews)}\n")
                break

        print(f"{cnt}th try\nNum. of reviews: {len(reviews)}\n")
        time.sleep(sleep + random.uniform(-sleep+0.1, 1))
        params['cursor'] = cursor.encode()

        # params['num_per_page'] = min(100, n)
        # n -= 100

        response = get_reviews(appid, params)

        # Error handling (requests.exceptions.JSONDecodeError: Unexpected UTF-8 BOM (decode using utf-8-sig): line 1 column 1 (char 0))
        if response == None:
            pass

        else:
            if 'reviews' in response and len(response['reviews']) > 0: # If there are reviews
                # print(f"len reviews: {len(response['reviews'])} ")

                cursor = response['cursor']

                # Filter review based on timestamp
                if (start_timestamp == None) and (end_timestamp == None):
                    reviews += response['reviews']

                else: # if (start_timestamp != None) or (end_timestamp != None):
                    if end_timestamp == None: # If there is no end_timestamp, set end_timestamp as now
                        end_timestamp = int(datetime.now().timestamp())
                    if start_timestamp == None: # If there is no start_timestamp, set start_timestamp as 0
                        start_timestamp = 0

                    for review in response["reviews"]:
                        if start_timestamp <= review["timestamp_created"] <= end_timestamp:
                            reviews.append(review)

                # empty_response_count = 0

            else:
                # print(f"len reviews: {len(response['reviews']) } ")
                print("Empty or error response encountered. Empty response count:", empty_response_count)
                empty_response_count += 1


        # Check the change of num. of reviews
        if len(reviews) == previous_reviews_num: # If the num. of reviews hasn't been changed
            empty_response_count += 1
            print(f"empty_respons_count: {empty_response_count}/{max_empty_responses}")

        else:
            empty_response_count = 0

        # Update previous_reviews_num
        previous_reviews_num = len(reviews)
        # cursor = response['cursor']

        # Break if there are too many consecutive empty responses
        if empty_response_count >= max_empty_responses:
            print("Stopping after multiple empty responses.")
            break

        # if len(response['reviews']) == 0:
        #     empty_reponse_count =
        #     break

    return reviews

In [ ]:
# 1. Get Reviews
reviews = get_n_reviews(appid='2358720', language="korean", num_try=None, start_date='2025-01-01', end_date='2025-01-10', sleep=0.2)
# reviews = get_n_reviews(appid='2358720', language="english", num_try=None, start_date='2025-01-01', end_date='2025-01-10', sleep=0.2)
# reviews = get_n_reviews(appid='2358720', language="schinese", num_try=None, start_date=None, end_date=None, sleep=0.2)
# reviews = get_n_reviews(appid='2358720', language="schinese", num_try=5, start_date='2024-10-30', end_date='2024-10-31', sleep=1)
        # appid: 게임 페이지 번호 (Black Myth: Wukong은 2358720; https://store.steampowered.com/appreviews/2358720 혹은 https://steamcommunity.com/app/2358720/reviews/ 참고)
        # language: korean, english, schinese, ...
        # num_try: 크롤링 횟수 제한하고 싶은 경우에 횟수만큼 입력. 입력하지 않으면, 발견되는 모든 리뷰를 크롤링. 1회 시도에 리뷰 20개씩 획득.
        # start_date, end_date: 시작 및 끝 날짜 설정. 'YYYY-MM-DD' 형식으로 써야 함.
        # sleep: 각 시도 중간의 딜레이. 기본은 1초 내외.

print(len(reviews))

1th try
Num. of reviews: 0

empty_respons_count: 1/5
2th try
Num. of reviews: 0

empty_respons_count: 2/5
3th try
Num. of reviews: 0

empty_respons_count: 3/5
4th try
Num. of reviews: 0

empty_respons_count: 4/5
5th try
Num. of reviews: 0

empty_respons_count: 5/5
6th try
Num. of reviews: 0

empty_respons_count: 6/5
7th try
Num. of reviews: 0

8th try
Num. of reviews: 1

empty_respons_count: 1/5
9th try
Num. of reviews: 1

empty_respons_count: 2/5
10th try
Num. of reviews: 1

empty_respons_count: 3/5
11th try
Num. of reviews: 1

empty_respons_count: 4/5
12th try
Num. of reviews: 1

empty_respons_count: 5/5
13th try
Num. of reviews: 1

empty_respons_count: 6/5
14th try
Num. of reviews: 1

empty_respons_count: 7/5
15th try
Num. of reviews: 1

empty_respons_count: 8/5
16th try
Num. of reviews: 1

empty_respons_count: 9/5
17th try
Num. of reviews: 1

empty_respons_count: 10/5
18th try
Num. of reviews: 1

empty_respons_count: 11/5
19th try
Num. of reviews: 1

empty_respons_count: 12/5
20th 

In [12]:
# 2. Get CSV
all_colums = list(reviews[0].keys())

result_df = pd.DataFrame(columns=all_colums)
    # colums: [recommendationid, author, language, review, timestamp_created, timestamp_updated, voted_up, votes_up, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, hidden_in_steam_china, steam_china_location, primarily_steam_deck]

for i in tqdm( range(len(reviews)) ):
    for j in range(len(all_colums)):
        result_df.loc[i, all_colums[j]] = str(reviews[i][all_colums[j]])

result_df.to_csv("steam_reviews.csv", index=False)

100%|██████████| 19/19 [00:00<00:00, 1392.85it/s]
